# Capstone: Financial Research Copilot

## Step 0: Setup and Configuration
Importing necessary libraries (yfinance, pandas, google-generativeai) and configuring the Google Gemini client using the API key from `.env`.

In [60]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import google.generativeai as genai
import json
from ddgs import DDGS

In [61]:
# 1. Load environment variables
load_dotenv()

# 2. Get API Key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Error: GOOGLE_API_KEY not found in .env file.")
else:
    print("API Key loaded successfully.")
    
    # 3. Configure the client
    try:
        genai.configure(api_key=api_key)
        print("Google GenAI Client configured successfully.")
    except Exception as e:
        print(f"Error configuring client: {e}")

API Key loaded successfully.
Google GenAI Client configured successfully.


## Step 1.1: Price History Tool
Defining `get_asset_price_history` to fetch OHLCV data for Stocks and ETFs using `yfinance`. This tool returns structured JSON data that the agent can analyze.


In [62]:
from typing import Dict, Any

def get_asset_price_history(symbol: str, asset_type: str = "equity", period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch historical OHLCV price data for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL', 'BTC-USD').
        asset_type (str): Type of asset ('equity', 'crypto', 'etf'). Defaults to 'equity'.
        period (str): Data period to download (e.g., '1mo', '3mo', '1y'). Defaults to '1mo'.
        
    Returns:
        dict: Structured data containing status and price history.
    """
    try:
        # Handle crypto symbols for yfinance (usually needs -USD suffix if not provided)
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
            symbol = f"{symbol}-USD"
            
        ticker = yf.Ticker(symbol)
        
        # Fetch history
        hist = ticker.history(period=period)
        
        if hist.empty:
            return {
                "status": "error",
                "message": f"No data found for {symbol}. Check ticker or period."
            }
            
        # Format data for the agent (convert to list of dicts)
        # We limit to the last 60 records to keep context small if period is long
        data_records = []
        for date, row in hist.tail(60).iterrows():
            data_records.append({
                "date": date.strftime('%Y-%m-%d'),
                "close": round(row['Close'], 2),
                "volume": int(row['Volume'])
            })
            
        return {
            "status": "success",
            "symbol": symbol,
            "currency": ticker.info.get('currency', 'USD'),
            "data": data_records
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [63]:
# # --- Testing the tool  ---
# print("Testing Tool 1.1 with AAPL...")
# result = get_asset_price_history("AAPL", period="5d")
# result = json.dumps(result, indent=4)
# print(result)


## Step 1.2: Fundamentals Tool
Defining `get_asset_fundamentals` to retrieve key financial metrics (P/E, Market Cap, Sector) using `yfinance`. This helps the agent assess value and risk.


In [64]:
def get_asset_fundamentals(symbol: str, asset_type: str = "equity") -> Dict[str, Any]:
    """
    Fetch fundamental data (P/E, Market Cap, Sector, Summary) for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL').
        asset_type (str): 'equity' or 'etf'. (Crypto has limited fundamentals in yf).
        
    Returns:
        dict: Key fundamentals or error message.
    """
    try:
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
             symbol = f"{symbol}-USD"

        ticker = yf.Ticker(symbol)
        info = ticker.info
        
        # Extract only the most relevant fields to save context tokens
        fundamentals = {
            "symbol": symbol,
            "name": info.get("shortName", "Unknown"),
            "sector": info.get("sector", "Unknown"),
            "industry": info.get("industry", "Unknown"),
            "market_cap": info.get("marketCap", "N/A"),
            "trailing_pe": info.get("trailingPE", "N/A"),
            "forward_pe": info.get("forwardPE", "N/A"),
            "dividend_yield": info.get("dividendYield", "N/A"),
            "fifty_two_week_high": info.get("fiftyTwoWeekHigh", "N/A"),
            "fifty_two_week_low": info.get("fiftyTwoWeekLow", "N/A"),
            "summary": info.get("longBusinessSummary", "No summary available")[:300] + "..." # Truncate summary
        }
        
        return {
            "status": "success",
            "data": fundamentals
        }
        
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [65]:
# # --- Test the tool immediately ---
# print("Testing Tool 1.2 with MSFT...")
# fund_result = get_asset_fundamentals("MSFT")
# fund_result = json.dumps(fund_result, indent=4)
# print(fund_result)

## Step 1.3: Technical Analysis Tool
Defining `calculate_technical_indicators` to compute Volatility (Standard Deviation) and Moving Averages. This gives the agent quantitative data to assess risk.


In [66]:
def calculate_technical_indicators(symbol: str, period: str = "3mo") -> Dict[str, Any]:
    """
    Calculate technical indicators (Volatility, SMA) for a symbol.
    
    Args:
        symbol (str): Ticker symbol.
        period (str): Data lookback period (default '3mo').
        
    Returns:
        dict: Calculated metrics including annualized volatility and current trend.
    """
    try:
        # We reuse our first tool logic to get the raw dataframe usually, 
        # but calling yfinance directly here is cleaner for the standalone tool.
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period)
        
        if hist.empty:
             return {"status": "error", "message": "No data for calculations."}
        
        # 1. Calculate Daily Returns
        hist['Returns'] = hist['Close'].pct_change()
        
        # 2. Annualized Volatility (Standard Deviation of returns * sqrt(252 trading days))
        volatility = hist['Returns'].std() * np.sqrt(252)
        
        # 3. Simple Moving Average (50-day) - requiring enough data
        if len(hist) >= 50:
            sma_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
        else:
            sma_50 = "Not enough data"
            
        current_price = hist['Close'].iloc[-1]
        
        # Determine trend
        trend = "Neutral"
        if isinstance(sma_50, (int, float)):
            if current_price > sma_50:
                trend = "Uptrend (Above 50d SMA)"
            else:
                trend = "Downtrend (Below 50d SMA)"
        
        return {
            "status": "success",
            "symbol": symbol,
            "current_price": round(current_price, 2),
            "annualized_volatility": round(volatility * 100, 2), # as percentage
            "sma_50": round(sma_50, 2) if isinstance(sma_50, float) else sma_50,
            "trend": trend
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}


In [67]:
# # --- Test the tool ---
# print("Testing Tool 1.3 with NVDA...")
# tech_result = calculate_technical_indicators("NVDA")
# tech_result = json.dumps(tech_result, indent=4)
# print(tech_result)

## Step 1.4: Market Opportunity Scanner Tool
Defining `scan_market_opportunities` to filter a watchlist of major assets for potential "Buys". It looks for:
1. **Value**: Low P/E ratio (< 25).
2. **Momentum**: Strong recent performance.
3. **Dip Buys**: Assets down significantly from their 52-week high.


In [68]:
def search_market_trends(query: str) -> Dict[str, Any]:
    """
    Search the web for financial news, trending stocks, or market analysis.
    Use this to find NEW ticker ideas (e.g., "undervalued AI stocks 2025").
    
    Args:
        query (str): Search query (e.g., "best biotech stocks under $50").
        
    Returns:
        dict: Top 5 search results with titles, snippets, and URLs.
    """
    try:
        results = []
        # specific 'news' backend is often good for markets, but 'text' is broader
        with DDGS() as ddgs:
            # Fetch top 5 results
            ddg_results = list(ddgs.text(query, max_results=5))
            
            if not ddg_results:
                return {"status": "error", "message": "No results found."}
            
            for r in ddg_results:
                results.append({
                    "title": r.get('title'),
                    "snippet": r.get('body'), # DDGS uses 'body' for snippet
                    "link": r.get('href')
                })
                
        return {
            "status": "success",
            "query": query,
            "results": results
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [69]:
# # --- Test ---
# print("Searching for Trending Stocks...")
# print(search_market_trends("fastest growing AI stocks 2025"))

## Step 2: Initialize Agent with Tools
We define the `tools` list containing our three functions and configure the Gemini Pro model to use them. We then start a chat session (Agent) that knows about these tools.


In [70]:
# 1. Define the Tool List
# We pass the actual function objects to the GenAI SDK. 
# The SDK automatically inspects the docstrings to understand how to use them.
my_tools = [
    get_asset_price_history,
    get_asset_fundamentals,
    calculate_technical_indicators
]

# 2. Initialize the Model with Tools
# We use 'gemini-1.5-flash' or 'gemini-1.5-pro' for best tool performance.
# 'auto' function calling mode is default, meaning the model decides when to use tools.
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash', # Or 'gemini-1.5-pro' if you have access
    tools=my_tools
)

# 3. Create the Agent (Chat Session)
# enable_automatic_function_calling=True means the SDK handles the loop:
# Model asks for tool -> SDK runs tool -> SDK gives answer back to Model -> Model answers user.
agent = model.start_chat(enable_automatic_function_calling=True)



In [71]:
# print("Agent initialized with Financial Tools!")

# # --- Quick Test ---
# # We ask a question that REQUIRES data to answer.
# print("\nAsking Agent: 'What is the current volatility of Apple and is it in an uptrend?'")
# response = agent.send_message("What is the current volatility of Apple (AAPL) and is it in an uptrend?")

# print("\nAgent Answer:")
# print(response.text)

## Step 3: Defining the Financial Advisor Persona
We re-initialize the agent with a **System Instruction**. This tells the model to act as a professional Investment Advisor, mandates that it *must* use tools before answering, and defines risk rules (e.g., "High volatility > 30% requires a warning").


In [72]:
# Define the System Prompt (Persona & Rules)
advisor_instruction = advisor_instruction = """
You are an expert Financial Research Assistant and Investment Advisor.

YOUR GOAL:
Help the user analyze assets and construct portfolios based on data.

RULES:
1. ALWAYS fetch real data using your tools before answering. Do not guess.
2. If a user asks about a stock, get its Fundamentals AND Technicals.
3. RISK ASSESSMENT:
   - If Annualized Volatility is > 30%, label the asset as "High Risk".
   - If Volatility is < 15%, label it as "Stable/Low Risk".
4. RECOMMENDATIONS: Cite specific metrics to support your view.

FORMATTING REQUIREMENTS:
- You MUST present data in two distinct tables:
  
  **Table 1: Fundamentals**
  | Metric | Value |


  **Table 2: Technicals**
  | Metric | Value |

- Make sure the table is well format in a uniform spaced manner with its borders
- Do not leave tables empty. If data is missing, write "N/A".
"""

# Re-initialize model with the system instruction
advisor_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    tools=my_tools,
    system_instruction=advisor_instruction
)

# Start the Advisor Chat
advisor_agent = advisor_model.start_chat(enable_automatic_function_calling=True)

print("Advisor Agent initialized with System Instructions!")


Advisor Agent initialized with System Instructions!


In [73]:
# # --- Test the Persona ---
# query = "Analyze Tesla (TSLA). Is it safe for a conservative retiree?"
# print(f"\nUser: {query}")
# response = advisor_agent.send_message(query)

# print("\nAdvisor Response:")
# print(response.text)

## Step 4: Portfolio Allocation Tool
Defining `allocate_portfolio` to simulate investment execution. This tool takes a list of assets and a budget, fetches real-time prices, and calculates an equal-weighted allocation (share counts and values).


In [74]:
MEMORY_FILE = "user_portfolio.json"

def allocate_portfolio(tickers: list[str], total_amount: float, weights: Dict[str, float] = None) -> Dict[str, Any]:
    """
    Create a portfolio allocation AND save it to memory immediately.
    """
    try:
        if not tickers or total_amount <= 0:
            return {"status": "error", "message": "Invalid tickers or amount."}
        
        # --- [Logic remains the same: 1. Strategy, 2. Loop] ---
        # ... (Copying the logic from previous step for brevity) ...
        
        strategy_name = "Custom Risk-Adjusted"
        if not weights:
            strategy_name = "Equal Weight"
            w_val = 1.0 / len(tickers)
            weights = {t: w_val for t in tickers}
        
        allocation = []
        total_spent = 0.0
        
        for symbol in tickers:
            # Heuristics...
            search_symbol = symbol
            if symbol in ["BTC", "ETH"] or (symbol.isupper() and len(symbol) <= 4 and "USD" not in symbol and symbol not in ["AAPL", "MSFT", "TSLA", "NVDA", "GOOG", "AMZN"]): 
                 pass
            
            ticker_obj = yf.Ticker(search_symbol)
            try:
                price = ticker_obj.fast_info['last_price']
            except:
                hist = ticker_obj.history(period="1d")
                price = hist['Close'].iloc[-1] if not hist.empty else 0.0
            
            if price > 0:
                asset_budget = total_amount * weights.get(symbol, 0.0)
                shares = round(asset_budget / price, 4)
                cost = round(shares * price, 2)
                allocation.append({
                    "symbol": symbol,
                    "price": round(price, 2),
                    "shares": shares,
                    "value": cost,
                    "weight_allocated": f"{round((cost / total_amount) * 100, 1)}%"
                })
                total_spent += cost
                
        remaining_cash = round(total_amount - total_spent, 2)
        
        result = {
            "status": "success",
            "strategy": strategy_name,
            "total_investment": total_amount,
            "allocation": allocation,
            "total_spent": round(total_spent, 2),
            "remaining_cash": remaining_cash
        }
        
        # --- NEW: AUTO-SAVE TO JSON ---
        # We manually trigger the save logic here so the Agent doesn't have to.
        portfolio_data = {
            "cash": remaining_cash,
            "total_value": total_amount,
            "holdings": {item['symbol']: item['shares'] for item in allocation}
        }
        
        with open(MEMORY_FILE, 'w') as f:
            json.dump(portfolio_data, f, indent=2)
            
        print(f"💾 [System] Portfolio automatically saved to {MEMORY_FILE}")
        return result

    except Exception as e:
        return {"status": "error", "message": str(e)}


In [75]:
# # --- Test ---
# print("Testing Portfolio Tool...")
# print(allocate_portfolio(["AAPL", "MSFT"], 1000, weights={"AAPL": 0.7, "MSFT": 0.3}))

## Step 5: Final Agent Assembly
Updating the agent with the `allocate_portfolio` tool and refined instructions. The agent can now Research, Analyze Risk, AND Construct Portfolios in a single conversation.


In [76]:
# # --- FINAL AGENT SETUP (Production Ready) ---

# # 1. Consolidate all tools
# final_tools = [
#     get_asset_price_history,        # Tool 1.1
#     get_asset_fundamentals,         # Tool 1.2
#     calculate_technical_indicators, # Tool 1.3
#     allocate_portfolio,             # Tool 1.4 (Portfolio)
#     search_market_trends            # Tool 1.5 (Web Search)
# ]

# # 2. robust System Instruction
# final_instruction = """
# You are a Strategic Investment Advisor (Powered by Gemini 2.5).

# YOUR GOAL:
# Build a solid core portfolio AND identify high-potential "Sleeper" stocks.

# WORKFLOW:
# 1. **Discovery**: Search for trending/underrated ideas (`search_market_trends`).
# 2. **Verification**: Check data for ALL candidates (`get_asset_fundamentals`, `calculate_technical_indicators`).
# 3. **Selection**:
#    - Select the BEST 3-5 assets for the **Main Portfolio** (Balance of Risk/Reward).
#    - Select 1-2 assets that are interesting but didn't make the main cut (e.g., higher risk, smaller cap, turnaround play) for the **"Underrated"** list.

# OUTPUT SECTIONS (Strictly follow this order):
# 1. **Market Insight**: "Based on search trends, I focused on [Sector]..."
# 2. **Core Portfolio Analysis**:
#    - Data Tables (Fundamentals & Technicals) for the main picks with spaced formating for better readability.
#    - Brief descrition on the choicess made.
# 3. **Proposed Allocation**: The `allocate_portfolio` table.
# 4. **💎 Underrated Stocks to Consider**:
#    - List 1-2 stocks that are NOT in the main portfolio above.
#    - **Why?**: Explain the "Growth Story" vs "Why it was excluded" (e.g., "High potential due to AI news, but excluded from core due to 85% volatility. Good for a small speculative bet.").
# 5. **Glossary**: Define technical terms.

# DISCLAIMER: "This is a simulation for educational purposes."
# """

# # Initialize Model with the specific version
# # Ensure your SDK version supports this model name string.
# final_model = genai.GenerativeModel(
#     model_name='gemini-2.5-flash',  # <--- UPDATED HERE
#     tools=final_tools,
#     system_instruction=final_instruction
# )



In [77]:
# 4. Start Session
# analyst_desk = final_model.start_chat(enable_automatic_function_calling=True)

# print("Final 'Trend Hunter' Analyst Desk is Ready!")
# print("Type 'exit' to stop.\n")

# # --- INTERACTIVE LOOP ---
# while True:
#     user_input = input("👤 You: ")
#     if user_input.lower() in ["exit", "quit"]:
#         break
    
#     print("Agent is thinking...")
#     try:
#         response = analyst_desk.send_message(user_input)
#         print(f"\nAnalyst:\n{response.text}\n")
#         print("-" * 80)
#     except Exception as e:
#         print(f"Error: {e}")

In [78]:
MEMORY_FILE = "user_portfolio.json"

def get_portfolio_state() -> Dict[str, Any]:
    """
    Retrieve the current user portfolio from memory.
    """
    try:
        if not os.path.exists(MEMORY_FILE):
            return {"status": "empty", "message": "No existing portfolio found."}
            
        with open(MEMORY_FILE, 'r') as f:
            data = json.load(f)
            # Basic validation
            if not isinstance(data, dict):
                return {"status": "error", "message": "Corrupt memory file."}
            return {"status": "success", "portfolio": data}
    except Exception as e:
        return {"status": "error", "message": str(e)}

In [79]:
def update_portfolio_memory(action: str, details: Dict[str, Any]) -> Dict[str, Any]:
    """
    Update the portfolio memory.
    Args: action ('deposit_cash', 'set_portfolio'), details (dict)
    """
    try:
        # Load existing or init
        if os.path.exists(MEMORY_FILE):
            with open(MEMORY_FILE, 'r') as f:
                portfolio = json.load(f)
        else:
            portfolio = {"cash": 0.0, "holdings": {}, "total_value": 0.0}
            
        # Logic
        if action == "deposit_cash":
            amount = float(details.get('amount', 0))
            portfolio['cash'] += amount
            portfolio['total_value'] += amount # Simple heuristic
            msg = f"Deposited ${amount}. Total Cash: ${portfolio['cash']}"
            
        elif action == "set_portfolio":
            # This expects the output format of 'allocate_portfolio'
            # e.g. details = {'remaining_cash': 500, 'allocation': [...]}
            portfolio['cash'] = details.get('remaining_cash', portfolio['cash'])
            portfolio['total_value'] = details.get('total_investment', 0) + portfolio['cash']
            
            # Convert allocation list to holdings dict for easier storage
            new_holdings = {}
            if 'allocation' in details:
                for item in details['allocation']:
                    sym = item['symbol']
                    shares = item.get('shares', 0)
                    new_holdings[sym] = shares
            
            portfolio['holdings'] = new_holdings
            msg = "Portfolio plan saved to memory."

        # Save
        with open(MEMORY_FILE, 'w') as f:
            json.dump(portfolio, f, indent=2)
            
        return {"status": "success", "message": msg, "current_state": portfolio}

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [ ]:
# --- MASTER AGENT SETUP ---

# 1. Consolidate ALL tools
final_tools = [
    get_asset_price_history,        # Tool 1.1
    get_asset_fundamentals,         # Tool 1.2
    calculate_technical_indicators, # Tool 1.3
    allocate_portfolio,             # Tool 1.4 (Auto-saving version)
    search_market_trends,           # Tool 1.5 (Web)
    get_portfolio_state             # Tool 1.6 (Read Memory)
    # Note: We removed 'update_portfolio_memory' because Tool 1.4 does it automatically now!
]

# 2. The Master System Instruction
final_instruction = """
You are a Strategic Investment Advisor (Powered by Gemini 2.5).

YOUR GOAL:
Manage the user's EVOLVING portfolio while identifying high-potential "Sleeper" stocks.

WORKFLOW:
1. **Check Memory First**: 
   - Call `get_portfolio_state`.
   - **If Empty**: Say "This looks like your first investment! Let's build a foundation."
   - **If Exists**: 
     - Calculate the *Current Real-Time Value* of their holdings (fetch current prices).
     - Report: "Your current portfolio is worth ~$X (Original: $Y)."
     - Use this context for new suggestions (e.g., "You are heavy on Tech, so let's diversify").

2. **Discovery Phase (Web)**:
   - If the user asks for ideas, call `search_market_trends`.
   - Focus on "Underrated" or "High Growth" keywords if the user wants aggressive growth.

3. **Verification Phase (Data)**:
   - Check Fundamentals & Technicals for ALL candidates (Core + Underrated).
   - Filter out assets with fatal flaws (e.g. bankruptcies) unless explicitly speculative.

4. **Selection**:
   - Select the BEST 3-5 assets for the **Main Portfolio**.
   - Select 1-2 assets for the **"💎 Underrated / Watchlist"** section (High risk/reward).

OUTPUT SECTIONS (Strictly follow this order):
1. **💰 Portfolio Status**: 
   - Current Value vs Cost Basis (if applicable).
   - "This looks like your first investment" (if new).

2. **🌍 Market Insight**: 
   - "Based on search trends, I focused on [Sector]..."

3. **📊 Core Portfolio Analysis**:
   - **Fundamentals Table** (P/E, Market Cap).
   - **Technicals Table** (Volatility, Trend).
   - *Brief description of why these choices were made.*

4. **📋 Proposed Allocation**: 
   - The `allocate_portfolio` table (Price | Shares | Value | Weight).

5. **💎 Underrated Stocks to Consider**:
   - List 1-2 stocks NOT in the main portfolio.
   - **Why?**: "Growth Story" vs "Why excluded" (e.g. "High potential but 85% volatility").

6. **Glossary**: Define technical terms used above.

DISCLAIMER: "This is a simulation for educational purposes."
"""

# 3. Initialize Model
final_model = genai.GenerativeModel(
    model_name='gemini-2.5-flash', 
    tools=final_tools,
    system_instruction=final_instruction
)

analyst_desk = final_model.start_chat(enable_automatic_function_calling=True)

print("🚀 Master Analyst Desk is Ready!")
print("Type 'exit' to stop.\n")

# --- INTERACTIVE LOOP ---
while True:
    user_input = input("👤 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    
    try:
        print(f"User :{user_input}\n")
        print("Agent is thinking...")
        response = analyst_desk.send_message(user_input)
        print(f"\nAnalyst:\n{response.text}\n")
    except Exception as e:
        print(f"Error: {e}")


🚀 Master Analyst Desk is Ready!
Type 'exit' to stop.



User :I have 10000 usd and i would like to inverst in both crypto and stocks assist me make the best portfolio

⏳ Agent is thinking...
💾 [System] Portfolio automatically saved to user_portfolio.json

🤖 Analyst:
| Asset     | Price       | Shares    | Value      | Weight  |
| :-------- | :---------- | :-------- | :--------- | :------ |
| **GOOGL** | $320.18     | 9.37      | $2,999.99  | 30.0%   |
| **AVGO**  | $402.96     | 6.20      | $2,500.00  | 25.0%   |
| **ALB**   | $129.99     | 11.54     | $1,499.99  | 15.0%   |
| **BTC-USD** | $90,916.51  | 0.022     | $2,000.16  | 20.0%   |
| **ETH-USD** | $2,994.64   | 0.33      | $999.91    | 10.0%   |

💎 Underrated Stocks to Consider:

*   **Cardano (ADA-USD):**
    *   **Growth Story:** Cardano is a blockchain platform known for its research-driven approach and focus on security and scalability. It aims to provide a more sustainable and interoperable ecosystem for dApps. While currently in a downtrend with high volatility, its strong deve